# Accretion disk

In this tutorial we will set up an accretion disk, which is a radiation source separate from the star, and add its contribution to the signal. The accretion disk we will use here (Mitsuda et al. 1984, Makishima et al. 1986) is a superposition of multicolor blackbody rings. It does not include relativistic effects and spectral hardening.

## Setting up the disk model

We follow here the description given by Makishima et al. (1986). The disk is parametrized by an inner cut-off radius, $R_\rm{in}$, a temperature at this radius, $T_\rm{in}$, and an outer radius, $R_\rm{out}$. The outer rings will contribute much less to the X-ray spectrum than inner rings due to being much colder, so the exact value of $R_\rm{out}$ is much less consequential than $R_\rm{in}$. For each ring, the temperature at some radius $T(r)$ is determined by the (gravitational) energy release associated with accretion towards a central object of mass $M_*$. It is expressed as:

$$T(r) = \left(\frac{3 G \dot{M} M_*}{8 \pi \sigma r^3}\right)^{1/4},$$

where $G$ is the gravitational constant, $\dot{M}$ is the accretion rate and $\sigma$ is the Stefan-Boltzmann constant. The surface area $dA$ of an infinitesimal ring between between $r-dr$ and $r$ is given by $2 \pi r dr$. The specific luminosity $dL_\rm{ring}(E)$ (energy/time/energy) of two hemispheres becomes

$$dL_\rm{ring}(E) = 4 B(E,T(r)) dA \int d\Omega= 8 \pi B(E,T(r)) r dr \int d\Omega = 8 \pi^2 B(E,T(r)) r dr.$$ 

Here the integral of the solid angle over one hemisphere evaluates to $\pi$. We multiply by 2 for the two sides of the disk and another factor of 2 appears since we define the $B(E, T)$, as half of the blackbody spectral radiance,  

$$B(E,T) = \frac{E^3}{c^2h^2}\left[\exp{\left(\frac{E}{k_\rm{B}T}\right)-1}\right]^{-1}. $$

Next up, we will provide the python functions involved in computing the radiation from the accretion disk that we will later collect into a python class. Let's start with definition of $B(E, T)$:

In [1]:
import numpy as np
from xpsi.global_imports import  _keV, _k_B, _h_keV
_c = 2.99792458E8
_c_cgs = _c*1E2

def B_E(self, E, T):
    '''
    Half the energy radiance of a blackbody. If the the photon energy is much higher than kT, 
    the radiance is set to zero and ignores an error

    parameters:
        E in keV
        T in keV

    returns:
        B_E in keV/s/keV/cm^2/sr (you will integrate over keV)
    '''
    with np.errstate(over='ignore'):
        safe_ET = np.float128(E / T)
        exp_safe_ET = np.exp(safe_ET)
    B = np.where(np.isinf(exp_safe_ET), 0, E**3 / (_h_keV**3 * _c_cgs**2) / (exp_safe_ET - 1))
    return B

/=============================================\
| X-PSI: X-ray Pulse Simulation and Inference |
|---------------------------------------------|
|                Version: 2.2.7               |
|---------------------------------------------|
|      https://xpsi-group.github.io/xpsi      |
\=============================================/

Check your emcee installation.
Check your installation of emcee if using the EnsembleSampler
Imported PyMultiNest.
Check your UltraNest installation.
Check your installation of UltraNest if using the UltranestSampler
Imported GetDist version: 1.5.3
Imported nestcheck version: 0.2.1


The specific flux of the disk $f_\rm{disk}(E)$ (energy/time/length$^2$/energy) is the integral of the contribution of each ring from $R_\rm{in}$ to $R_\rm{out}$. But, this can be conveniently expressed as an integral over $dT$:


$$f_\rm{disk}(E) = \frac{\cos{i} \int dL_\rm{ring}}{4\pi D^2}=\frac{2 \pi \cos{i}}{D^2}\int_{R_\rm{in}}^{R_\rm{out}} r B(E,T(r)) dr = \frac{8\pi R_\rm{in}^2 \cos{i}}{3 D^2 T_\rm{in}}\int_{T_\rm{out}}^{T_\rm{in}}\Big(\frac{T}{T_\rm{in}}\Big)^{-11/3}B(E,T) dT.$$

Where $D$ is the distance and we introduced $\cos{i}$ to scale the flux to account for the viewing angle.

In the code, similar to the approach in Xspec (The diskbb model), we collect the three parameters outside of the integral into a normalisation factor 
$$K_\rm{disk} = R_\rm{in}^2 \cos{i} / D^2.$$ 

In [2]:
def get_k_disk(cos_i, r_in, distance):
    """
    This function calculates the k-disk value for a given set of input parameters.
    
    Parameters
    ----------
    cos_i: The cosine inclination angle of the disk
    r_in: The inner radius of the disk in kilometers
    distance: The distance to the disk in kiloparsecs
    
    Returns
    -------
    k_disk normalisation factor
    
    """

    k_disk = cos_i * (r_in / (distance / 10))**2 # in [km/ 10 kpc]^2
    
    # K_disk is cos_i*R_in^2/D^2 in (km / 10 kpc)^2.
    # (1 km / 10 kpc)^2 = 1.0502650e-35 [ cm/cm ]^2

    k_disk *= 1.0502650e-35 # now k_disk is unitless

    return k_disk

We also define the integrand $l_\rm{disk}(E,T) = \frac{1}{T_\rm{in}}(\frac{T}{T_\rm{in}})^{-11/3}B(E,T)$.

In [3]:
from scipy.integrate import quad

def l_disk_integrand(self, T, E, T_in, spectral_radiance):
    '''
    parameters:
        T, T_in in keV
        E in keV

    returns:
        integrand in spectral radiance units/keV. This integrand will 
        be integrated over keV.
    '''

    integrand = (T/T_in)**(-11/3)*spectral_radiance(E, T)/T_in
    return integrand

def l_disk_integrated(self, E, T_in, T_out, spectral_radiance, epsrel):
    '''
    parameters:
        T, T_in in keV
        E in keV

    returns:
        disk luminosity [spectral radiance units]. 
    '''

    integrated,_= quad(self.l_disk_integrand, T_out, T_in, args=(E, T_in, spectral_radiance), epsrel=epsrel)
    return integrated

We obtain

$$f_\rm{disk}(E) = \frac{8 \pi}{3} K_\rm{disk} \int_{T_\rm{out}}^{T_\rm{in}} l_\rm{disk}(E,T) dT$$

In [6]:
k_B_over_keV = _k_B / _keV

def get_f_disk(self, energies):
    """ Evaluate f_disk(E). This function also requires the X-PSI parameters T_in and K_disk to be defined.
    
    Parameters
    ----------
    energies[keV]
    
    Returns
    -------
    f_disk [keV/s/cm^2/keV]
    
    """
    
    T_in = self['T_in'] # in 10^T K
    K_disk = self['K_disk'] # 

    T_in_keV = k_B_over_keV * pow(10.0, T_in)       
    T_out_keV = T_in_keV*1e-1 # hardcode a factor 10 difference
    
    epsrel = 1e-4 # hardcode precision for integration

    f_disk_array = np.array([])
    for energy in energies:
        f_disk_value = self.l_disk_integrated(energy, T_in_keV, T_out_keV, spectral_radiance, epsrel) 
        f_disk_array=np.append(f_disk_array,f_disk_value)
    
    f_disk_array *=K_disk*8*np.pi/3 # keV/s/cm^2/keV
    
    return f_disk_array

To obtain the photon flux for the disk instead of energy flux, one can divide $f_\rm{disk}(E)$ by the photon energy. Let's collect these function into an X-PSI compatible class. With initiation of an instance of the class, we initiate the three parameters that control the accretion disk. When we call the instance, we compute the photon flux. 

In [5]:
from xpsi.ParameterSubspace import ParameterSubspace
from xpsi.Parameter import Parameter, Derive

class Disk(ParameterSubspace):
    
    def __init__(self, bounds=None, values=None):

        doc = """
        Temperature at inner disk radius in log10 Kelvin.
        """
        inner_temperature = Parameter('T_in',
                                strict_bounds = (3., 10.),
                                bounds = bounds.get('T_in', None),
                                doc = doc,
                                symbol = r'$T_{in}$',
                                value = values.get('T_in', None))

        doc = """
        Disk R_in in kilometers.
        """
        inner_radius = Parameter('R_in',
                                strict_bounds = (0., 1e3),
                                bounds = bounds.get('R_in', None),
                                doc = doc,
                                symbol = r'$R_{in}$',
                                value = values.get('R_in', None))

        
        doc = """
        Disk normalisation cos_i*R_in^2/D^2.
        """
        background_normalisation = Parameter('K_disk',
                                strict_bounds = (0., 1e100),
                                bounds = bounds.get('K_disk', None),
                                doc = doc,
                                symbol = r'$K_{BB}$',
                                value = values.get('K_disk', None))
        

        super(Disk, self).__init__(inner_temperature, inner_radius, background_normalisation)

    def __call__(self, energies):
        spectral_radiance = self.B_E 
        self.disk_flux = self.get_f_disk(energies, spectral_radiance)/energies
        return self.disk_flux

Disk.B_E = B_E
Disk.get_k_disk = get_k_disk
Disk.l_disk_integrand = l_disk_integrand
Disk.l_disk_integrated = l_disk_integrated
Disk.get_f_disk = get_f_disk